In [1]:
from modules.modules import openBrowserCrefisa, get_tables, work_tables
import datetime
import os
import pandas as pd

In [18]:
# classe para conferir a quantidade de contratos da mesma data
class contracts_conference():

    def __init__(self, date_work = datetime.date):
        self.process = False
        self.date= date_work 
        self.load_new_table()
        self.load_table()
        
        
    # Carrega os novos relatórios
    def load_new_table(self):
        path_new_reports =  f"./relatorios_novos/{self.date.year}/{self.date.month}"
        path_to_read = path_new_reports + f"/relatorio_pagamentos{self.date.day}_{self.date.month}_{self.date.year}.xls" 
        os.makedirs(path_new_reports, exist_ok = True)
        if os.path.exists(path_to_read):
            dados = pd.read_html(path_to_read, header = 0, thousands='.')
            dados_lidos = pd.DataFrame(dados[0])
            self.process = True
            setattr(self, 'dados_novos', dados_lidos)
    
    # Carrega os relatórios usados no storm
    
    def load_table(self):
        path_new_reports =  f"./relatorios/{self.date.year}/{self.date.month}"
        path_to_read = path_new_reports + f"/relatorio_{self.date.day}_{self.date.month}_{self.date.year}.xls" 
        if self.process:
            if os.path.exists(path_to_read):
                dados = pd.read_html(path_to_read, header = 0, thousands='.')
                dados_lidos = pd.DataFrame(dados[0])
                setattr(self, 'dados', dados_lidos)


    # Verifica se são as mesmas propostas
    def retorna_dimensao(self):
        path_to_save = f'../Pagamentos_atrasados/{self.date.year}/{self.date.month}'
        save_date = datetime.date.today() - datetime.timedelta(days = 1)
        path_to_save_file = path_to_save + f"/pagamento_atrasado_{save_date}.csv"
        os.makedirs(path_to_save, exist_ok = True)
        if self.process:
            dados_novos = getattr(self, 'dados_novos')
            dados = getattr(self, 'dados')
            propostas_novas = dados_novos.NUMERO_ADE.map(lambda x: x.replace("'", ""))
            propostas = dados.NUMERO_ADE.map(lambda x: x.replace("'", ""))
            result = list(set(propostas_novas) - set(propostas))
            dados_novos['NUMERO_ADE'] = dados_novos['NUMERO_ADE'].map(lambda x: x.replace("'", ""))
            filtrados = dados_novos[dados_novos['NUMERO_ADE'].isin(result)]
            print("Propostas pagas em atraso foram encontradas!")
            if os.path.exists(path_to_save_file):
                filtrados = pd.concat([filtrados, pd.read_csv(path_to_save_file)])
            filtrados.to_csv(path_to_save_file, index = False)

# lendo_dados = contracts_conference(date_work = datetime.date(2024, 3, 25)).retorna_dimensao()
    

In [19]:
lendo_dados = contracts_conference(date_work = datetime.date(2024, 4, 2)).retorna_dimensao()

Propostas pagas em atraso foram encontradas!
